Analisis exploratorio: 

Como primer paso cargamos los dataframes para su analisis.

In [ ]:
import pandas as pd
from db_config import engine 



df_ordenes = pd.read_sql('SELECT * FROM fact_ordenes', con=engine)
df_detalle_ordenes = pd.read_sql('SELECT * FROM fact_detalle_ordenes', con=engine)
df_usuarios = pd.read_sql('SELECT * FROM dim_usuarios', con=engine)
df_categorias = pd.read_sql('SELECT * FROM dim_categorias', con=engine)
df_productos = pd.read_sql('SELECT * FROM dim_productos', con=engine)
df_metodos_pago = pd.read_sql('SELECT * FROM dim_metodos_pago', con=engine)
df_direcciones_envio = pd.read_sql('SELECT * FROM dim_direcciones_envio', con=engine)
df_carrito = pd.read_sql('SELECT * FROM fact_carrito', con=engine)
df_ordenes_metodo_pago = pd.read_sql('SELECT * FROM fact_ordenes_metodo_pago', con=engine)
df_resenas_productos = pd.read_sql('SELECT * FROM fact_resenas_productos', con=engine)
df_historial_pagos = pd.read_sql('SELECT * FROM fact_historial_pagos', con=engine)



Detección de valores nulos
El siguiente scrip detecta valores nulos o extraños que representen vacios, la implementacion se realiza sobre todos los dataframes al mismo tiempo ya que previamente sabemos que no hay valores nulos.

In [17]:
def find_weird_NAN_representations(df: pd.DataFrame) -> None:
    nan_representations = ["NA", "N/A", "null", "NULL", "nan", "NaN", "", " ", "?"]
    columns = []
    for column in df.columns:
        # Convertir a string para comparar correctamente
        col_as_str = df[column].astype(str)
        if col_as_str.isin(nan_representations).any():
            columns.append(column)
    print(f"NAN representation found for columns {columns}" if columns else "No NAN representation found in columns")


# implementación

find_weird_NAN_representations(df_ordenes)
find_weird_NAN_representations(df_detalle_ordenes)
find_weird_NAN_representations(df_usuarios)
find_weird_NAN_representations(df_categorias)
find_weird_NAN_representations(df_productos)
find_weird_NAN_representations(df_metodos_pago)
find_weird_NAN_representations(df_direcciones_envio)
find_weird_NAN_representations(df_carrito)
find_weird_NAN_representations(df_ordenes_metodo_pago)
find_weird_NAN_representations(df_resenas_productos)
find_weird_NAN_representations(df_historial_pagos)



No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns
No NAN representation found in columns


Analisis de porcentaje de nulos en los dataframes

In [ ]:

print(df_ordenes.isnull().sum() / len(df_ordenes) * 100)
print(df_detalle_ordenes.isnull().sum() / len(df_detalle_ordenes) * 100)
print(df_categorias.isnull().sum() / len(df_categorias) * 100)
print(df_usuarios.isnull().sum() / len(df_usuarios) * 100)
print(df_metodos_pago.isnull().sum() / len(df_metodos_pago) * 100)
print(df_productos.isnull().sum() / len(df_productos) * 100)
print(df_direcciones_envio.isnull().sum() / len(df_direcciones_envio) * 100)
print(df_carrito.isnull().sum() / len(df_carrito) * 100)
print(df_ordenes_metodo_pago.isnull().sum() / len(df_ordenes_metodo_pago) * 100)
print(df_resenas_productos.isnull().sum() / len(df_resenas_productos) * 100)
print(df_historial_pagos.isnull().sum() / len(df_historial_pagos) * 100)






Analisis por dataframe
dim_categorias
no es necesaria mayor validacion ya que los datos se encuentran correctos.

In [ ]:
print(df_categorias.head())

Analisis por dataframe
dim_metodos_pago
no es necesaria mayor validacion ya que los datos se encuentran correctos.

In [ ]:
print(df_metodos_pago.head())

Analisis por dataframe
dim_productos
analizaremos que los campos precio stock y categoria id sean datos numericos.


In [ ]:

from pandas import DataFrame

def find_weird_values_for_columns(df: DataFrame, numeric_columns: list[str]) -> DataFrame:
    copy_df = df.copy()
    columns = []
    masks = []
    for column in numeric_columns:
        copy_df[column] = copy_df[column].astype(str)
        mask = ~copy_df[column].str.match(r"^-?\d+\.?\d*$", na=True)
        if len(df[mask][column].unique()) > 0:
            columns.append(column)
            masks.append(df[mask][column].unique())
    return DataFrame({"Column": columns, "Weird_Values": masks})

find_weird_values_for_columns(df_productos, ["stock","precio","categoriaid"])

,Column,Weird_Values


Analisis por dataframe
dim_usuarios
vemos que los campos tienen acentos corregimos y volvemos a cargar la data a la base de datos.


In [ ]:

import pandas as pd
import unicodedata
from sqlalchemy import create_engine

print(df_usuarios.head())

# Función para eliminar acentos
def remove_accents(text):
    if isinstance(text, str):
        nfkd_form = unicodedata.normalize('NFKD', text)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
    return text

# Aplicar la limpieza a todo el DataFrame
def clean_dataframe_accents(df):
    return df.applymap(remove_accents)


# Limpiar el DataFrame
df_clean = clean_dataframe_accents(df_usuarios)


# Guardar el DataFrame limpio en la base de datos
try:
    df_clean.to_sql('dim_usuarios', con=engine, if_exists='replace', index=False)
    print("✅ Datos insertados correctamente.")
except Exception as e:
    print("❌ Error al insertar:")
    print(e)



C:\Users\emiliano.abregu\AppData\Local\Temp\ipykernel_7852\2022045011.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(remove_accents)


✅ Datos insertados correctamente.


Analisis fact_carrito nos aseguramos que los datos numericos sean 100% numericos.

In [31]:
find_weird_values_for_columns(df_carrito, ["usuario_id","producto_id","cantidad"])

,Column,Weird_Values


Analisis fact_detalle_ordenes nos aseguramos que los datos numericos sean 100% numericos. 

In [32]:
find_weird_values_for_columns(df_detalle_ordenes, ["orden_id","producto_id","cantidad","precio_unitario"])

,Column,Weird_Values


Analisis dim_direcciones_envio vemos que los campos tienen acentos corregimos y volvemos a cargar la data a la base de datos.

In [ ]:
df_clean = clean_dataframe_accents(df_direcciones_envio)


# Guardar el DataFrame limpio en la base de datos
try:
    df_clean.to_sql('dim_direcciones_envio', con=engine, if_exists='replace', index=False)
    print("✅ Datos insertados correctamente.")
except Exception as e:
    print("❌ Error al insertar:")
    print(e)

C:\Users\emiliano.abregu\AppData\Local\Temp\ipykernel_7852\2022045011.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(remove_accents)


✅ Datos insertados correctamente.


Analisis fact_historial_pagos nos aseguramos que los datos numericos sean 100% numericos.  

In [34]:
find_weird_values_for_columns(df_historial_pagos, ["orden_id","metodo_pago_id","monto"])

,Column,Weird_Values


Analisis fact_ordenes nos aseguramos que los datos numericos sean 100% numericos.   

In [37]:
find_weird_values_for_columns(df_ordenes, ["total"])

,Column,Weird_Values


Analisis fact_ordenes_metodo_pago nos aseguramos que los datos numericos sean 100% numericos.    

In [38]:
find_weird_values_for_columns(df_ordenes_metodo_pago, ["monto_pagado","metodo_pago_id"])

,Column,Weird_Values
